# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235269424813                   -0.50    8.0
  2   -7.250218249675       -1.83       -1.40    1.0
  3   -7.251290784589       -2.97       -2.05    4.0
  4   -7.251297812971       -5.15       -2.23    5.0
  5   -7.251337965312       -4.40       -3.03    4.0
  6   -7.251338727299       -6.12       -3.66    2.0
  7   -7.251338772625       -7.34       -3.94    3.0
  8   -7.251338791939       -7.71       -4.24    2.0
  9   -7.251338798593       -8.18       -5.33    1.0
 10   -7.251338798689      -10.02       -5.41    4.0
 11   -7.251338798700      -10.94       -5.73    2.0
 12   -7.251338798704      -11.38       -6.26    2.0
 13   -7.251338798705      -12.44       -6.99    3.0
 14   -7.251338798705      -13.75       -7.46    3.0
 15   -7.251338798705   +  -14.57       -7.84    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05564312041311757
[ Info: Arnoldi iteration step 2: normres = 0.6369107891332869
[ Info: Arnoldi iteration step 3: normres = 0.7200114725092105
[ Info: Arnoldi iteration step 4: normres = 0.28653990945541297
[ Info: Arnoldi iteration step 5: normres = 0.3520717837814289
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.31e-02, 6.42e-02, 2.64e-01, 2.22e-01, 1.89e-02)
[ Info: Arnoldi iteration step 6: normres = 0.42635148306671344
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.55e-03, 1.01e-01, 3.83e-01, 1.15e-01, 6.73e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07298401620500555
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.71e-04, 1.09e-02, 2.34e-02, 3.00e-02, 5.50e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11861492035654068
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.79e